# XGBoost
## Alex Chen

$var_1$ = Open

$var_2$ = Close

# Import Library

In [1]:
import warnings

# Suppressing warnings
warnings.filterwarnings("ignore")

#library pengolahan data
import datetime as dt
import pandas as pd
from pandas_datareader import data as pdr
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import pandas_ta as ta

#library sklearn untuk modelling.
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

#library error
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

# library xgboost
from xgboost import XGBRegressor


# Tarik data

In [2]:
yf.pdr_override()
df = pdr.get_data_yahoo(["WLD-USD"])

[*********************100%***********************]  1 of 1 completed


In [3]:
df.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-05-26,4.708926,4.738072,4.498226,4.559780,4.559780,179073708
2024-05-27,4.535849,4.864353,4.515802,4.806207,4.806207,259875984
2024-05-28,4.806207,4.806207,4.547240,4.661622,4.661622,283685354
2024-05-29,4.661622,4.972356,4.651304,4.889364,4.889364,485522697
2024-05-30,4.931377,5.025631,4.707520,4.930467,4.930467,491429184


In [4]:
df.shape

(806, 6)

# Edit Data sesuai research

In [5]:
df_xg = df.iloc[:,:4]
days=5
df1 = pd.DataFrame()
for j in range(days+1):
    for i in df_xg.columns:
        if j==0:
            df1[f'{i}'] = df_xg[i].iloc[j:]
        else:
            df1[f'{i}_t-{j}'] = df_xg[i].shift(j)

In [6]:
df1.loc[:,['Open']+list(df1.columns[4:])].tail(5)

,Open,Open_t-1,High_t-1,Low_t-1,Close_t-1,Open_t-2,High_t-2,Low_t-2,Close_t-2,Open_t-3,...,Low_t-3,Close_t-3,Open_t-4,High_t-4,Low_t-4,Close_t-4,Open_t-5,High_t-5,Low_t-5,Close_t-5
Date,,,,,,,,,,,,,,,,,,,,,
2024-05-26,4.708926,4.589289,4.807861,4.577909,4.707670,4.693564,4.757608,4.481779,4.589283,4.973073,...,4.508942,4.693564,5.092682,5.207076,4.917881,4.973073,5.071708,5.221206,4.942319,5.092682
2024-05-27,4.535849,4.708926,4.738072,4.498226,4.559780,4.589289,4.807861,4.577909,4.707670,4.693564,...,4.481779,4.589283,4.973073,5.003993,4.508942,4.693564,5.092682,5.207076,4.917881,4.973073
2024-05-28,4.806207,4.535849,4.864353,4.515802,4.806207,4.708926,4.738072,4.498226,4.559780,4.589289,...,4.577909,4.707670,4.693564,4.757608,4.481779,4.589283,4.973073,5.003993,4.508942,4.693564
2024-05-29,4.661622,4.806207,4.806207,4.547240,4.661622,4.535849,4.864353,4.515802,4.806207,4.708926,...,4.498226,4.559780,4.589289,4.807861,4.577909,4.707670,4.693564,4.757608,4.481779,4.589283
2024-05-30,4.931377,4.661622,4.972356,4.651304,4.889364,4.806207,4.806207,4.547240,4.661622,4.535849,...,4.515802,4.806207,4.708926,4.738072,4.498226,4.559780,4.589289,4.807861,4.577909,4.707670


# Train Test Split Data

In [7]:
X = df1.iloc[-400:,]
X = X.loc[:,list(df1.columns[4:])]
y = df1.iloc[-400:,3:4]

In [8]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.3,shuffle=True,random_state=33)

In [9]:
x_train.columns

Index(['Open_t-1', 'High_t-1', 'Low_t-1', 'Close_t-1', 'Open_t-2', 'High_t-2',
       'Low_t-2', 'Close_t-2', 'Open_t-3', 'High_t-3', 'Low_t-3', 'Close_t-3',
       'Open_t-4', 'High_t-4', 'Low_t-4', 'Close_t-4', 'Open_t-5', 'High_t-5',
       'Low_t-5', 'Close_t-5'],
      dtype='object')

# Bangkitkan Model

In [11]:
mod2 = XGBRegressor(n_estimators=100, learning_rate = 0.1,objective='reg:squarederror',max_depth=9,min_child_weight=1,subsample=0.9,colsample_bytree=0.9)

mod2.fit(x_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.9, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=9, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

# Evaluasi Model

In [12]:
print(mod2.score(x_train,y_train))
print(mod2.score(x_test,y_test))

#mod2.predict(x_test)

0.9999886290906648
0.9728728366942457


In [14]:
prediction = mod2.predict(X.tail(30).values)

print(mean_absolute_error(y.tail(30),prediction))
mean_absolute_percentage_error(y.tail(30),prediction)

0.14347286224365235


0.028301075472690288

In [15]:
X = df1.iloc[-400:,]
X = X.loc[:,list(df1.columns[4:])]
y = df1.iloc[-400:,3:4]

In [16]:
n=300
prediction = mod2.predict(X.tail(n).values)
open_10 = df.Open.tail(n)
index_df = y.tail(n).index
real_10 = y.tail(n).values.flatten()
pred_10 = prediction.flatten()

In [32]:
result_df = pd.DataFrame({'y_real':real_10,'y_pred':pred_10,'open_price':open_10},index=index_df)
result_df['action'] = np.where(result_df.y_pred>result_df.open_price,1,-1)

In [34]:
result_df

,y_real,y_pred,open_price,action
Date,,,,
2023-08-05,2.130008,2.128466,2.169348,-1
2023-08-06,2.041331,2.251726,2.130136,1
2023-08-07,2.039499,2.039499,2.041177,-1
2023-08-08,1.936297,1.936994,2.039137,-1
2023-08-09,1.832185,1.835684,1.936634,-1
...,...,...,...,...
2024-05-26,4.559780,4.561898,4.708926,-1
2024-05-27,4.806207,4.617728,4.535849,1
2024-05-28,4.661622,4.659579,4.806207,-1


**Kita akan mencoba membuat trading strategy, di mana posisi kita akan berubah setiap hari.**

Jika Open > y_pred maka kita akan membuka posisi short sell dan jika Open < y_pred maka kita akan membuka posisi long buy dengan asumsi tanpa fee dan komisi

In [36]:
buy_df = result_df.loc[result_df.action==1]
buy_df['pnl_percent'] = (buy_df['y_real'] - buy_df['open_price'])/buy_df['open_price']
buy_df['G/L'] = np.where(buy_df['pnl_percent']>0, "Gain",'Loss')

buy_df

,y_real,y_pred,open_price,action,pnl_percent,G/L
Date,,,,,,
2023-08-06,2.041331,2.251726,2.130136,1,-0.041690,Loss
2023-08-11,1.739924,1.739396,1.679649,1,0.035885,Gain
2023-08-12,1.760038,1.758129,1.740497,1,0.011227,Gain
2023-08-13,1.769442,1.772002,1.759679,1,0.005548,Gain
2023-08-18,1.543943,1.543429,1.366970,1,0.129464,Gain
...,...,...,...,...,...,...
2024-05-20,5.071791,5.061521,4.700936,1,0.078890,Gain
2024-05-24,4.589283,4.791500,4.693564,1,-0.022218,Loss
2024-05-25,4.707670,4.705857,4.589289,1,0.025795,Gain


In [37]:
sell_df = result_df.loc[result_df.action==-1]
sell_df['pnl_percent'] =  (-sell_df['y_real'] + sell_df['open_price'])/sell_df['y_real']
sell_df['G/L'] = np.where(sell_df['pnl_percent']>0, "Gain",'Loss')

sell_df

,y_real,y_pred,open_price,action,pnl_percent,G/L
Date,,,,,,
2023-08-05,2.130008,2.128466,2.169348,-1,0.018469,Gain
2023-08-07,2.039499,2.039499,2.041177,-1,0.000823,Gain
2023-08-08,1.936297,1.936994,2.039137,-1,0.053112,Gain
2023-08-09,1.832185,1.835684,1.936634,-1,0.057008,Gain
2023-08-10,1.679517,1.697474,1.832469,-1,0.091069,Gain
...,...,...,...,...,...,...
2024-05-22,4.973073,4.973923,5.092682,-1,0.024051,Gain
2024-05-23,4.693564,4.700274,4.973073,-1,0.059552,Gain
2024-05-26,4.559780,4.561898,4.708926,-1,0.032709,Gain


In [46]:
buy_df.groupby("G/L").count() / buy_df.shape[0]

,y_real,y_pred,open_price,action,pnl_percent
G/L,,,,,
Gain,0.862595,0.862595,0.862595,0.862595,0.862595
Loss,0.137405,0.137405,0.137405,0.137405,0.137405


In [47]:
sell_df.groupby("G/L").count() / sell_df.shape[0]

,y_real,y_pred,open_price,action,pnl_percent
G/L,,,,,
Gain,0.828402,0.828402,0.828402,0.828402,0.828402
Loss,0.171598,0.171598,0.171598,0.171598,0.171598


## Model yang dihasilkan memperoleh untung yang lebih banyak daripada kerugian.